In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from linearmodels import PanelOLS
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import ResidualsPlot
from scipy import stats

In [2]:
joint = pd.read_excel('joint_snap.xlsx')

In [3]:
#convert the month column into correct form
joint['Month'] = pd.to_datetime(joint['Month'])
age = (joint['Month'] - joint['Inception Date'])/np.timedelta64(1,'M')
age = [int(i) for i in age]
joint['Age'] = age
#joint.dropna(subset=["esg_score"], inplace=True)

#convert the esb_l from string to float
esb_l = pd.DataFrame({'Equity Style Box (Long)':['Large Value', 'Large Growth', 'Large Blend', 'Mid Value', 'Mid Growth', 'Mid Blend','Small Value','Small Growth','Small Blend'], 'esb_l': [1, 2, 3, 4, 5, 6, 7, 8, 9]})
joint_new = joint.join(esb_l.set_index('Equity Style Box (Long)'), on='Equity Style Box (Long)')

#relative_flow is the size divided by flow value
flow = joint_new["Fund Flow"]
size = joint_new['Fund Size']
relative_flow = flow/size
joint_new['relative_flow'] = relative_flow

joint_new = joint_new.rename(columns={"Fund Sustainability": "esg_score"})

In [13]:
#build a new column, in which the relative_flow is the value of last month
capm_org = joint_new['performance_Three_Factor']
capm_nxt = [np.nan]*len(capm_org)

verschiebung = 1

for i in range(3276):
    for j in range(verschiebung,92):
         capm_nxt[92*i+j-verschiebung] = capm_org[92*i+j]
            
joint_new['capm_Performance_mod'] = capm_nxt

In [14]:
#build a new column, performance * esg_score
#performance_score_1 = [joint_new['capm_Performance'][i] * joint_new['esg_score'] for i in range(len(joint_new))]
performance_score_1 = joint_new['performance_Three_Factor']*joint_new['esg_score']
joint_new['esg_performance'] = performance_score_1

In [15]:
#catagorize the month - 2012/01 to 2019/08, totoally 92 months
month = pd.Categorical(joint_new.Month)

#use the 'Month' to divide the dataset, the index is Fund ID
df = joint_new.set_index(['FundId', 'Month'])
df['Month'] = month

In [18]:
#capm Performance summary
exog_vars = ['esg_score','size_log', 'Age', 'performance_Three_Factor', 'esg_performance', 'relative_flow']
exog = sm.add_constant(df[exog_vars])

mod = PanelOLS(df.capm_Performance_mod, exog, entity_effects=True, time_effects=True)

res = mod.fit()

In [19]:
print(res)

                           PanelOLS Estimation Summary                            
Dep. Variable:     capm_Performance_mod   R-squared:                        0.0001
Estimator:                     PanelOLS   R-squared (Between):             -0.0421
No. Observations:                239246   R-squared (Within):              -0.0023
Date:                  Thu, Apr 09 2020   R-squared (Overall):             -0.0025
Time:                          09:17:51   Log-likelihood                -5.255e+05
Cov. Estimator:              Unadjusted                                           
                                          F-statistic:                      33.334
Entities:                          3269   P-value                           0.0000
Avg Obs:                         73.186   Distribution:                F(1,235886)
Min Obs:                         1.0000                                           
Max Obs:                         91.000   F-statistic (robust):             33.334
    